# Annealing by Toshiba SBM

In [3]:
import subprocess
import joblib

In [1]:
ip="ip address for server"
cmd=f"""
curl -i -H "Content-Type: application/octet-stream" -X POST "{ip}:8000/solver/ising?steps=0&loops=10" --data-binary "@test.mm"
"""
cmd=cmd.replace("\n","")

In [7]:
#test job
%time res=subprocess.check_output(cmd, shell=True)

CPU times: user 4.21 ms, sys: 13.3 ms, total: 17.6 ms
Wall time: 75 ms


In [8]:
res

b'HTTP/1.1 200 OK\r\nContent-Type: application/json; charset=utf-8\r\nContent-Length: 136\r\nETag: W/"88-weJtY/4/+o83WSvs5lD0i8l81Ak"\r\nDate: Thu, 19 Aug 2021 08:22:43 GMT\r\nProxy-Connection: Keep-Alive\r\nConnection: Keep-Alive\r\n\r\n{"id":"r1664466134","time":0.02,"wait":0,"runs":3200,"steps":8,"C":0.014653,"dt":1,"message":"finished","value":-193,"result":[1,0,1,1]}'

# Make qubo for SBM

In [154]:
#make sparse dict from array-type qubo
original_qubo=joblib.load("../data/rbm_J.bin")

scale=10**5
dim=original_qubo.shape[0]
sparse_dict={}
for i in range(dim):
    for j in range(dim):
        val=-original_qubo[i][j]*scale
        if val!=0:
            sparse_dict[f"{j+1} {i+1}"]=int(val)

In [156]:
#save as mm file
num_non_zero_params=len(sparse_dict.keys())

output=f"""%%MatrixMarket matrix coordinate real symmetric
{dim} {dim} {num_non_zero_params}
"""

#output
for k,v in sparse_dict.items():
    output+=f"{k} {v}\n"
    
with open("qubo.mm","w") as f:
    f.write(output[:-1])

# Annealing by SBM

In [4]:
timeout=60
cmd=f"""
curl -i -H "Content-Type: application/octet-stream" -X POST "{ip}:8000/solver/ising?steps=0&loops=10&timeout={timeout}" --data-binary "@qubo.mm"
"""

cmd=cmd.replace("\n","")

In [73]:
#%time res=subprocess.check_output(cmd, shell=True)

CPU times: user 1.29 ms, sys: 5.56 ms, total: 6.85 ms
Wall time: 1min 2s


In [74]:
print(res)

b'HTTP/1.1 100 Continue\r\n\r\nHTTP/1.1 200 OK\r\nContent-Type: application/json; charset=utf-8\r\nContent-Length: 4136\r\nETag: W/"1028-MwZccDhIkqhiMFV4+Jh9KwaZ5qo"\r\nDate: Thu, 19 Aug 2021 08:28:46 GMT\r\nProxy-Connection: Keep-Alive\r\nConnection: Keep-Alive\r\n\r\n{"id":"r2814050749","time":60.07,"wait":0,"runs":2240,"steps":8001,"C":0.0000066,"dt":1,"message":"timeout","value":-3961787,"result":[0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0